In [3]:
!export MYSQLCLIENT_LDFLAGS="-L"
!export MYSQLCLIENT_CFLAGS="-I"

In [8]:
!pip install mysql-connector-python faker awsglue-local

ERROR: Ignored the following versions that require a different python version: 0.9.1 Requires-Python >=2.7,<3.0; 1.0.2 Requires-Python >=3,<3.8
ERROR: Could not find a version that satisfies the requirement awsglue-local (from versions: none)
ERROR: No matching distribution found for awsglue-local

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [78]:

import sys
import random
from faker import Faker
from datetime import datetime, timedelta
import mysql.connector
from mysql.connector import errorcode
#from awsglue.utils import getResolvedOptions


fake = Faker()

In [79]:

cnx = mysql.connector.connect(
    host='db2.c0oohyxv9j5t.us-east-1.rds.amazonaws.com',
    port='3306',
    user='admin',
    password='Test1234',
    database='gluedb'
)
cur = cnx.cursor()

In [13]:
def create_table(cursor):
    event_table_description = (
        "CREATE TABLE `sport_event` ("
        "  `event_id` INT NOT NULL AUTO_INCREMENT,"
        "  `sport_type` VARCHAR(256) NOT NULL,"
        "  `start_date` DATETIME NOT NULL,"
        "  `location` VARCHAR(256) NOT NULL,"
        " `creation_time` DATETIME NOT NULL,"
        " `updated_time` DATETIME NOT NULL,"
        "  PRIMARY KEY (`event_id`)"
        
        ") ENGINE=InnoDB"
    )
    try:
        print("Creating table {}: ".format("sport_event"), end='')
        cursor.execute(event_table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

    ticket_table_description = (
        "CREATE TABLE `ticket` ("
        "  `ticket_id` INT NOT NULL AUTO_INCREMENT,"
        "  `event_id` INT NOT NULL,"
        "  `seat_level` VARCHAR(256) NOT NULL,"
        "  `seat_location` VARCHAR(256) NOT NULL,"
        "  `ticket_price` INT NOT NULL,"
        "  PRIMARY KEY (`ticket_id`),"
         " `creation_time` DATETIME NOT NULL,"
        " `updated_time` DATETIME NOT NULL,"
        "  FOREIGN KEY (`event_id`) REFERENCES sport_event(`event_id`)"
        ") ENGINE=InnoDB"
    )
    try:
        print("Creating table {}: ".format("ticket"), end='')
        cursor.execute(ticket_table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


    customer_table_description = (
        "CREATE TABLE `customer` ("
        "  `customer_id` INT NOT NULL AUTO_INCREMENT,"
        "  `customer_name` VARCHAR(256) NOT NULL,"
        "  `email_address` VARCHAR(256) NOT NULL,"
        "  `phone_number` VARCHAR(256) NOT NULL,"
         " `creation_time` DATETIME NOT NULL,"
        " `updated_time` DATETIME NOT NULL,"
        "  PRIMARY KEY (`customer_id`)"
        ") ENGINE=InnoDB"
    )
    try:
        print("Creating table {}: ".format("customer"), end='')
        cursor.execute(customer_table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


    ticket_activity_table_description = (
        "CREATE TABLE `ticket_activity` ("
        "  `ticket_id` INT NOT NULL,"
        "  `purchased_by` INT NOT NULL,"
        "  `created_at` DATETIME NOT NULL,"
        "  `updated_at` DATETIME NOT NULL,"
        "  PRIMARY KEY (`ticket_id`),"
        "  FOREIGN KEY (`ticket_id`) REFERENCES ticket(`ticket_id`),"
        "  FOREIGN KEY (`purchased_by`) REFERENCES customer(`customer_id`)"
        ") ENGINE=InnoDB"
    )
    try:
        print("Creating table {}: ".format("ticket_activity"), end='')
        cursor.execute(ticket_activity_table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


In [27]:
sport_type_list = [
    "Baseball",
    "Football"
]

location_list = [
    "Seattle, US",
    "New York, US",
    "San Francisco, US",
    "Los Angeles, US",
    "Boston, US",
    "Chicago, US"
]

In [28]:
def generate_seat_list(num):
    seat_level_price = {
        "Standard": 100,
        "Premium": 300
    }
    list = []
    for seat_level in seat_level_price.keys():
        for i in range(num):
            list.append((seat_level, f"{seat_level[0]}-{i}", seat_level_price[seat_level]))
    return list

In [67]:
a = datetime.today()
a.time()

datetime.time(3, 24, 55, 407748)

In [68]:

def initialize_data(cursor, connection):
    today = datetime.today()
    time = today.time()

    try:
        print("Initializing data in table `sport_event` and `ticket`:", end='')
        add_event = ("INSERT INTO sport_event "
                     "(sport_type, start_date, location,creation_time,updated_time) "
                     "VALUES (%s, %s, %s,%s,%s)")
        for num in range(50):
            future_date = (today + timedelta(days=random.randint(30,365))).date()
            data_event = (random.choice(sport_type_list), future_date, random.choice(location_list),today.date(),today.date())
            cursor.execute(add_event, data_event)
            latest_event_id = cursor.lastrowid

            add_ticket = ("INSERT INTO ticket "
                          "(event_id, seat_level, seat_location, ticket_price, creation_time,updated_time) "
                          "VALUES (%s, %s, %s, %s,%s,%s)")
            for seat_level, seat_location, ticket_price in generate_seat_list(100):
                data_ticket = (latest_event_id, seat_level, seat_location, ticket_price, today.date(), today.date())
                cursor.execute(add_ticket, data_ticket)
    except mysql.connector.Error as err:
        print(err.msg)
    else:
        print("OK")

    try:
        print("Ingesting data in table `customer`:", end='')
        add_customer = ("INSERT INTO customer "
                        "(customer_name, email_address, phone_number, creation_time, updated_time) "
                        "VALUES (%s, %s, %s, %s, %s)")

        for num in range(1000):
            data_customer = (fake.name(), fake.ascii_email(), fake.phone_number(), today.time(), today.time())
            cursor.execute(add_customer, data_customer)
    except mysql.connector.Error as err:
        print(err.msg)
    else:
        print("OK")

    connection.commit()

In [69]:
initialize_data(cur, cnx)

Initializing data in table `sport_event` and `ticket`:OK
Ingesting data in table `customer`:OK


In [75]:
now = datetime.now()
id = 1
formatted_date = now.strftime('%Y-%m-%d %H:%M:%S')
print(formatted_date, now.date())
email_address = 'dedfdf@abc.com'
query = """update customer set email_address = %s, updated_time =%s where customer_id = %s"""
input_data=(email_address,formatted_date,1)
print(query,formatted_date)
cur.execute(query,input_data)
cnx.commit()
# Assuming you have a cursor named cursor you want to execute this query on:
#cursor.execute('insert into table(id, date_created) values(%s, %s)', (id, formatted_date))

2023-10-17 03:29:23 2023-10-17
update customer set email_address = %s, updated_time =%s where customer_id = %s 2023-10-17 03:29:23


In [76]:
cur.execute("select email_address,updated_time from customer where customer_id =1 ")
cur.fetchall()

[('dedfdf@abc.com', datetime.datetime(2023, 10, 17, 3, 29, 23))]

In [20]:



def ingest_data(cursor, connection):
    today = datetime.today()

    print("Selecting one record from table {}".format("ticket"))
    cursor.execute("SELECT ticket.ticket_id FROM ticket LEFT OUTER JOIN ticket_activity on ticket.ticket_id = ticket_activity.ticket_id WHERE ticket_activity.ticket_id is NULL ORDER BY RAND() LIMIT 1")
    rows = cursor.fetchall()
    ticket_id = 0
    for row in rows:
        ticket_id = row[0]

    print("Selecting one record from table {}".format("customer"))
    cursor.execute("SELECT customer_id FROM customer ORDER BY RAND() LIMIT 1")
    rows = cursor.fetchall()
    customer_id = 0
    for row in rows:
        customer_id = row[0]

    print("Ingesting data into table {}: ".format("ticket_activity"), end='')
    add_ticket_activity = ("INSERT INTO ticket_activity "
                 "(ticket_id, purchased_by, created_at, updated_at) "
                 "VALUES (%s, %s, %s, %s)")

    data_ticket_activity = (ticket_id, customer_id, today, today)
    cursor.execute(add_ticket_activity, data_ticket_activity)

    connection.commit()

mode='initial'
#if args['mode'] == 'initial':
create_table(cur)
initialize_data(cur, cnx)

ingest_data(cur, cnx)

cur.close()
cnx.close()


Creating table sport_event: already exists.
Creating table ticket: already exists.
Creating table customer: already exists.
Creating table ticket_activity: already exists.
Initializing data in table `sport_event` and `ticket`:OK
Ingesting data in table `customer`:OK
Selecting one record from table ticket
Selecting one record from table customer
Ingesting data into table ticket_activity: 

In [36]:
!aws glue reset-job-bookmark --job-name customer

{
    "JobBookmarkEntry": {
        "JobName": "customer",
        "Version": 9,
        "Run": 6,
        "Attempt": 0,
        "JobBookmark": ""
    }
}


## create one table to test

In [81]:
 customer_table_description = (
        "CREATE TABLE `demo1` ("
        "  `customer_id` INT NOT NULL AUTO_INCREMENT,"
        "  `customer_name` VARCHAR(256) NOT NULL,"
        "  `email_address` VARCHAR(256) NOT NULL,"
        "  `phone_number` VARCHAR(256) NOT NULL,"
         " `creation_time` DATETIME NOT NULL,"
        " `updated_time` DATETIME NOT NULL,"
        "  PRIMARY KEY (`customer_id`)"
        ") ENGINE=InnoDB"
    )
try:
        print("Creating table {}: ".format("customer"), end='')
        cur.execute(customer_table_description)
except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
else:
        print("OK")

Creating table customer: OK


In [89]:
today = datetime.today()
time = today.time()
now = datetime.now()
formatted_date = now.strftime('%Y-%m-%d %H:%M:%S')
add_customer = ("INSERT INTO demo1 "
                        "(customer_name, email_address, phone_number, creation_time, updated_time) "
                        "VALUES (%s, %s, %s, %s, %s)")

for num in range(5):
            data_customer = (fake.name(), fake.ascii_email(), fake.phone_number(), formatted_date, formatted_date)
            cur.execute(add_customer, data_customer)
cnx.commit()

### verify data

In [99]:
query="select * from demo1"
cur.execute(query)
cur.fetchall()

[(1,
  'Danielle Kelley',
  'xyz@abc.com',
  '+1-995-929-0153x419',
  datetime.datetime(2023, 10, 17, 13, 26, 45),
  datetime.datetime(2023, 10, 17, 15, 0, 36)),
 (2,
  'Natalie Daugherty',
  'danielware@hotmail.com',
  '001-831-460-2158x30256',
  datetime.datetime(2023, 10, 17, 13, 26, 45),
  datetime.datetime(2023, 10, 17, 13, 26, 45)),
 (3,
  'Joy Wade',
  'maryjohnson@acosta.com',
  '909-420-8608x930',
  datetime.datetime(2023, 10, 17, 13, 26, 45),
  datetime.datetime(2023, 10, 17, 13, 26, 45)),
 (4,
  'Maria Savage',
  'pbean@martin.com',
  '374-953-1731',
  datetime.datetime(2023, 10, 17, 13, 26, 45),
  datetime.datetime(2023, 10, 17, 13, 26, 45)),
 (5,
  'Dale Palmer',
  'kristinecole@king.info',
  '819.773.6614x98003',
  datetime.datetime(2023, 10, 17, 13, 26, 45),
  datetime.datetime(2023, 10, 17, 13, 26, 45))]

### update first row

In [100]:
now = datetime.now()
id = 1
formatted_date = now.strftime('%Y-%m-%d %H:%M:%S')
print(formatted_date, now.date())
email_address = 'new@abc.com'
query = """update demo1 set email_address = %s, updated_time =%s where customer_id = %s"""
input_data=(email_address,formatted_date,1)
print(query,formatted_date)
cur.execute(query,input_data)
cnx.commit()

2023-10-17 15:56:27 2023-10-17
update demo1 set email_address = %s, updated_time =%s where customer_id = %s 2023-10-17 15:56:27
